In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install stanza
!pip install textblob
!pip install tensorflow-gpu
!pip install tensorflow_hub
!pip install tqdm
!pip install bert-for-tf2

     |████████████████████████████████| 1.4MB 6.0MB/s 
     |████████████████████████████████| 890kB 49.3MB/s 
     |████████████████████████████████| 2.9MB 49.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=d0a3c135999a6aa93fd790e290760cdf0ce9cefa6b41c283b57efc47da52be72
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 235kB 5.7MB/s 
     |█████████████▋                  | 167.8MB 1.4MB/s eta 0:02:45ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/urllib3/response.py", line 425, in _error_catcher
    yield
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/urllib3/response.py", line 507, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/cachecontrol/filewrapper.py

In [3]:
import pandas as pd
import textblob
from textblob import TextBlob
import json
import tqdm
from tensorflow.keras import *
from transformers import BertTokenizer,TFBertModel
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import stanza
import bert
from collections import defaultdict
from sklearn.tree import DecisionTreeClassifier
import tensorflow_hub as hub
from sklearn.svm import SVC
from string import punctuation
import nltk
import regex as re
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
!cp "/content/drive/My Drive/toxic span/toxic_span_practice.csv" "./toxic_span_practice.csv"
!cp "/content/drive/My Drive/toxic span/toxic_span_train.csv" "./toxic_span_train.csv"
!cp "/content/drive/My Drive/toxic span/civil comments/selected_test.csv" "./civil_test.csv"
!cp "/content/drive/My Drive/toxic span/civil comments/selected_train.csv" "./civil_train.csv"
!cp "/content/drive/My Drive/toxic span/bad-words.csv" "./bad-words.csv"
!cp "/content/drive/My Drive/toxic span/stopwords.txt" "./stopwords.txt"

In [5]:
with open('stopwords.txt') as file:
    stopwords = {x.strip() for x in file.readlines()}

In [6]:
def f1(preds,trues):
    if len(trues) == 0:
        return 1. if len(preds) == 0 else 0.
    if len(preds) == 0:
        return 0.
    predictions_set = set(preds)
    gold_set = set(trues)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [7]:
def avg_f1(preds,trues):
    avg_f1_total = 0.0
    for pred,true in zip(preds,trues):
        avg_f1_total += f1(pred,true)
    return avg_f1_total/len(preds)

In [8]:
def clear_unwanted_parts_of_text(text):
    pattern = "[\w]+[\p{P}]*[\w]+"
    toks = re.findall(pattern,text)
    return " ".join(toks).lower()

## toxic span test

In [9]:
test_set = pd.read_csv("toxic_span_practice.csv")
test_set['spans'] = test_set['spans'].apply(lambda x : json.loads(x))
train_set = pd.read_csv("toxic_span_train.csv")
train_set['spans'] = train_set['spans'].apply(lambda x : json.loads(x))
toxic_span_dataset = test_set.append(train_set,ignore_index=True)
toxic_span_dataset['text'] = toxic_span_dataset['text'].apply(lambda x : x.lower())
toxic_span_dataset['original_text'] = toxic_span_dataset['text']
toxic_span_dataset['text'] = toxic_span_dataset['text'].apply(lambda x : clear_unwanted_parts_of_text(x))

## civil comments dataset

### create new train test

In [ ]:
tweet_dataset = pd.read_csv("./civil_comments.csv")
tweet_dataset['target'] = tweet_dataset['target'].apply(lambda x : 0 if x < 0.49 else 1)
tweet_false_dataset = tweet_dataset[tweet_dataset['target']==0]
tweet_true_dataset = tweet_dataset[tweet_dataset['target']==1]
max_of_each_class = min(len(tweet_false_dataset),len(tweet_true_dataset))
sampel_false = tweet_false_dataset.sample(n=max_of_each_class)
sampel_true = tweet_true_dataset.sample(n=max_of_each_class)
civil_dataset = pd.concat([sampel_false,sampel_true]).sample(frac=1)
civil_dataset['comment_text'].apply(lambda x: x.lower())
civil_dataset['tweet'] = civil_dataset['comment_text'].apply(clear_unwanted_parts_of_text)
civil_dataset = civil_dataset[['tweet','target']]

In [ ]:
train_dataset = civil_dataset.iloc[:,:int(len(civil_dataset)*0.8)]
validation_dataset = civil_dataset.iloc[:,int(len(civil_dataset)*0.8):]

NameError: ignored

### use selected train test

In [10]:
train_dataset = pd.read_csv("./civil_train.csv")
train_dataset['tweet'] = train_dataset['tweet'].astype('str')
validation_dataset = pd.read_csv("./civil_test.csv")
validation_dataset['tweet'] = validation_dataset['tweet'].astype('str')

In [ ]:
train_dataset

,Unnamed: 0,tweet,target
0,1773475,No The Rebel is really really really bad As sa...,1
1,732149,It'll be fun to watch Trump's reaction when hi...,0
2,1754948,Listen carefully to the new Canadians from Asi...,0
3,1005880,Because it impacts lower income people more th...,0
4,355553,Third party Loyalty programs are an expensive ...,0
...,...,...,...
231433,1379740,You need brain transplant,1
231434,442683,Are you even Canadian You are disagreeing with...,1
231435,1053364,Probably NanakuliBozo and his gang of misfits,1
231436,61661,Hal Shea is that nom de plume for Little Jack ...,0


In [ ]:
validation_dataset

,Unnamed: 0,tweet,target
0,624409,Because it was 97 MILLION numnut not BILLION,0
1,1543248,Slavery is so funny,1
2,1534723,Ah the old hypocritical you drive car right or...,1
3,297745,That's because they are great citizens who for...,0
4,1733710,It is wise to know what you are talking about ...,1
...,...,...,...
57855,1721460,60 students no wait THUGS holding thousands ho...,0
57856,1747537,Another way for Alberta to tackle its deficit ...,0
57857,1529935,Curious Any tax advantages to starting persona...,0
57858,1035768,But Comey’s testimony Thursday made clear that...,1


# Attention Model

In [11]:
class AttentionScoreLayer(layers.Layer):
    def __init__(self,units,layers_count = 2, **kwargs):
        assert layers_count > 1
        self.init = initializers.get('glorot_uniform')
        self.units = units
        self.hlayers_count = layers_count - 2
        self.hidden_layers = []
        self.hidden_biases = []
        super(AttentionScoreLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W1 = self.add_weight(shape=(input_shape[2],self.units),
                                initializer=self.init,
                                name='{}_W1'.format(self.name))
        for i in range(self.hlayers_count):
            self.hidden_layers.append(self.add_weight(shape=(self.units, self.units,),
                                    initializer=self.init,
                                    name=('{}_W'+str(i+2)).format(self.name)))
            self.hidden_biases.append(self.add_weight(shape=(1,self.units),
                                initializer=initializers.zeros(),
                                name=('{}_B'+str(i+2)).format(self.name)))
        self.W_out = self.add_weight(shape=(self.units,1),
                                initializer=self.init,
                                name='{}_W_out'.format(self.name))
        self.B1 = self.add_weight(shape=(1,self.units),
                                initializer=initializers.zeros(),
                                name='{}_B1'.format(self.name))
        self.B_out = self.add_weight(shape=(input_shape[1],),
                                initializer=initializers.zeros(),
                                name='{}_B3'.format(self.name))

        super(AttentionScoreLayer, self).build(input_shape)

    def call(self, x, mask=None):
        w = K.relu(K.dot(x,self.W1) + K.repeat(self.B1,x.shape[1]))
        for Wi,Bi in zip(self.hidden_layers,self.hidden_biases):
            w = K.relu(K.dot(w,Wi) + K.repeat(Bi,x.shape[1]))
        w = K.squeeze(K.dot(w,self.W_out),-1) + self.B_out
        return K.softmax(w)
        
        
    def get_output_shape_for(self, input_shape):
        return input_shape[0], input_shape[-1]

    def compute_output_shape(self, input_shape):
        """Shape transformation logic so Keras can infer output shape
        """
        return (input_shape[0], input_shape[-1],)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
                "units" : self.units,
                "hlayers_count" : self.hlayers_count,
        })
        return config

class AttentionWithContext(layers.Layer):

    def __init__(self,units, **kwargs):
        self.score = AttentionScoreLayer(units)
        self.units = units
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        super(AttentionWithContext, self).build(input_shape)

    def call(self, x, mask=None):
        ait = self.score(x)
        # ait = ait * ait
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        # sentence vector si is returned
        return K.sum(weighted_input, axis=1)

    def get_output_shape_for(self, input_shape):
        return input_shape[0], input_shape[-1]

    def compute_output_shape(self, input_shape):
        """Shape transformation logic so Keras can infer output shape
        """
        return (input_shape[0], input_shape[-1],)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
                "units" : self.units,
        })
        return config

In [12]:
def createBertwithSentenceLevelAttention(max_input_length,bert_type="L12",attention_layers=2,attention_size=4096):
    bert_layer = None
    if bert_type == "L12":
        bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2",trainable=False)
    else:
        bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/2",trainable=True)
    input_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_input_ids",dtype=tf.int32)
    input_type_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_token_type_ids",dtype=tf.int32)
    input_attention_mask_layer = layers.Input(shape=(max_input_length,),name="encoder_attention_mask",dtype=tf.int32)
    bert_output = bert_layer([input_ids_layer,input_attention_mask_layer,input_type_ids_layer])
    output = AttentionWithContext(attention_size)(bert_output[1])
    output = layers.Dropout(0.1)(output)
    output = layers.Dense(4096,activation="relu")(output)
    output = layers.Dropout(0.1)(output)
    output = layers.Dense(4096,activation="relu")(output)
    output = layers.Dense(2,activation="softmax")(output)
    model = models.Model(inputs=[input_ids_layer,input_type_ids_layer,input_attention_mask_layer],outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),loss="categorical_crossentropy",metrics=['accuracy'])
    return model

In [13]:
max_length = 400

In [14]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2",trainable=False)
# bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/2",trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
bert_tokenizer_instance = bert.bert_tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

def find_max_length(texts,tokenizer):
    max_length = 0
    for text in texts:
        ids_len = len(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text)))
        if ids_len > max_length:
            max_length = ids_len
    return max_length

def convert_text_to_number_and_fix_length(this_x, tokenizer_mod):
    this_x_ids = tokenizer_mod.convert_tokens_to_ids(this_x,)
    if len(this_x_ids) >= max_length:
        this_x_fix_len = this_x_ids[:max_length]
    else:
        this_x_fix_len = this_x_ids + [0]*(max_length - len(this_x_ids)) 
    return this_x_fix_len

def create_mask_for_input(this_x):
    if len(this_x) >= max_length:
        this_x_masks = [1]*max_length
    else:
        this_x_masks = [1]*len(this_x) + [0] * (max_length - len(this_x))
    return this_x_masks

def create_req_bert_inputs(all_X, tokenizer_mod):
    X_input_ids, X_input_masks, X_segment_ids = [], [], []

    for this_x in tqdm.tqdm(all_X, position=0, leave=True):
        X_input_ids.append(convert_text_to_number_and_fix_length(this_x, tokenizer_mod))
        X_input_masks.append(create_mask_for_input(this_x))
        X_segment_ids.append([0 for _ in range(max_length)])    
    return X_input_ids, X_input_masks, X_segment_ids

def clean_text(all_X, tokenizer_mod=None):
  converted_X = []
  for this_x in all_X:
    this_x_tokenized = tokenizer_mod.tokenize(this_x)
    converted_X.append(this_x_tokenized)
  return converted_X

INFO:absl:resolver HttpCompressedFileResolver does not support the provided handle.
INFO:absl:resolver GcsCompressedFileResolver does not support the provided handle.


In [15]:
model = createBertwithSentenceLevelAttention(max_length)

INFO:absl:resolver HttpCompressedFileResolver does not support the provided handle.
INFO:absl:resolver GcsCompressedFileResolver does not support the provided handle.


In [16]:
model.load_weights("/content/drive/MyDrive/toxic span/final saved models/attention/bert/attention")

In [28]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input_ids (InputLayer)  [(None, 400)]        0                                            
__________________________________________________________________________________________________
encoder_attention_mask (InputLa [(None, 400)]        0                                            
__________________________________________________________________________________________________
encoder_token_type_ids (InputLa [(None, 400)]        0                                            
__________________________________________________________________________________________________
keras_layer_4 (KerasLayer)      [(None, 768), (None, 109482241   encoder_input_ids[0][0]          
                                                                 encoder_attention_mask[0][0

In [17]:
train_data = clean_text(train_dataset['tweet'].to_list(),bert_tokenizer_instance)
input_ids,input_masks,input_segs = create_req_bert_inputs(train_data,bert_tokenizer_instance)
train_data = [np.array(input_ids),np.array(input_segs),np.array(input_masks)]
test_data = clean_text(validation_dataset['tweet'].to_list(),bert_tokenizer_instance)
input_ids,input_masks,input_segs = create_req_bert_inputs(test_data,bert_tokenizer_instance)
test_data = [np.array(input_ids),np.array(input_segs),np.array(input_masks)]
test_labels = utils.to_categorical(validation_dataset['target'].to_list())
train_labels = utils.to_categorical(train_dataset['target'].to_list())

100%|██████████| 57860/57860 [00:04<00:00, 11601.55it/s]


In [ ]:
model.fit(train_data,train_labels,validation_data=(test_data,test_labels),epochs=10,batch_size=32,callbacks=[callbacks.ModelCheckpoint("/content/drive/MyDrive/toxic span/final saved models/attention/bert/attention",save_weights_only=True)])

Epoch 1/10


 170/7233 [..............................] - ETA: 1:19 - loss: 0.6914 - accuracy: 0.53 - ETA: 34:26 - loss: 2.3712 - accuracy: 0.500 - ETA: 45:20 - loss: 3.1967 - accuracy: 0.458 - ETA: 50:47 - loss: 2.9622 - accuracy: 0.460 - ETA: 54:06 - loss: 2.5188 - accuracy: 0.462 - ETA: 56:17 - loss: 2.2774 - accuracy: 0.463 - ETA: 57:50 - loss: 2.1014 - accuracy: 0.464 - ETA: 58:59 - loss: 1.9486 - accuracy: 0.476 - ETA: 59:54 - loss: 1.8155 - accuracy: 0.486 - ETA: 1:00:37 - loss: 1.7035 - accuracy: 0.484 - ETA: 1:01:12 - loss: 1.6190 - accuracy: 0.471 - ETA: 1:01:41 - loss: 1.5429 - accuracy: 0.471 - ETA: 1:02:05 - loss: 1.4736 - accuracy: 0.488 - ETA: 1:02:26 - loss: 1.4155 - accuracy: 0.495 - ETA: 1:02:45 - loss: 1.3699 - accuracy: 0.493 - ETA: 1:03:01 - loss: 1.3299 - accuracy: 0.490 - ETA: 1:03:16 - loss: 1.2907 - accuracy: 0.496 - ETA: 1:03:28 - loss: 1.2563 - accuracy: 0.500 - ETA: 1:03:40 - loss: 1.2256 - accuracy: 0.504 - ETA: 1:03:50 - loss: 1.1979 - accuracy: 0.507 - ETA: 1:03:59 - 

 340/7233 [>.............................] - ETA: 1:05:30 - loss: 0.6124 - accuracy: 0.700 - ETA: 1:05:30 - loss: 0.6126 - accuracy: 0.700 - ETA: 1:05:30 - loss: 0.6113 - accuracy: 0.701 - ETA: 1:05:29 - loss: 0.6102 - accuracy: 0.702 - ETA: 1:05:29 - loss: 0.6097 - accuracy: 0.703 - ETA: 1:05:28 - loss: 0.6086 - accuracy: 0.703 - ETA: 1:05:28 - loss: 0.6075 - accuracy: 0.704 - ETA: 1:05:28 - loss: 0.6063 - accuracy: 0.705 - ETA: 1:05:27 - loss: 0.6054 - accuracy: 0.705 - ETA: 1:05:27 - loss: 0.6040 - accuracy: 0.705 - ETA: 1:05:26 - loss: 0.6032 - accuracy: 0.706 - ETA: 1:05:26 - loss: 0.6022 - accuracy: 0.706 - ETA: 1:05:25 - loss: 0.6026 - accuracy: 0.707 - ETA: 1:05:25 - loss: 0.6022 - accuracy: 0.707 - ETA: 1:05:25 - loss: 0.6013 - accuracy: 0.707 - ETA: 1:05:24 - loss: 0.6009 - accuracy: 0.708 - ETA: 1:05:24 - loss: 0.6010 - accuracy: 0.708 - ETA: 1:05:23 - loss: 0.5996 - accuracy: 0.708 - ETA: 1:05:23 - loss: 0.5987 - accuracy: 0.709 - ETA: 1:05:22 - loss: 0.5984 - accuracy: 0.7

 510/7233 [=>............................] - ETA: 1:04:08 - loss: 0.5463 - accuracy: 0.735 - ETA: 1:04:08 - loss: 0.5463 - accuracy: 0.735 - ETA: 1:04:07 - loss: 0.5464 - accuracy: 0.735 - ETA: 1:04:07 - loss: 0.5462 - accuracy: 0.735 - ETA: 1:04:06 - loss: 0.5457 - accuracy: 0.735 - ETA: 1:04:06 - loss: 0.5456 - accuracy: 0.735 - ETA: 1:04:05 - loss: 0.5452 - accuracy: 0.736 - ETA: 1:04:05 - loss: 0.5446 - accuracy: 0.736 - ETA: 1:04:04 - loss: 0.5444 - accuracy: 0.736 - ETA: 1:04:04 - loss: 0.5444 - accuracy: 0.736 - ETA: 1:04:03 - loss: 0.5445 - accuracy: 0.736 - ETA: 1:04:03 - loss: 0.5440 - accuracy: 0.737 - ETA: 1:04:02 - loss: 0.5436 - accuracy: 0.737 - ETA: 1:04:02 - loss: 0.5436 - accuracy: 0.737 - ETA: 1:04:01 - loss: 0.5432 - accuracy: 0.737 - ETA: 1:04:01 - loss: 0.5426 - accuracy: 0.738 - ETA: 1:04:00 - loss: 0.5423 - accuracy: 0.738 - ETA: 1:04:00 - loss: 0.5423 - accuracy: 0.738 - ETA: 1:03:59 - loss: 0.5418 - accuracy: 0.738 - ETA: 1:03:59 - loss: 0.5414 - accuracy: 0.7

 680/7233 [=>............................] - ETA: 1:02:38 - loss: 0.5179 - accuracy: 0.753 - ETA: 1:02:38 - loss: 0.5179 - accuracy: 0.753 - ETA: 1:02:37 - loss: 0.5181 - accuracy: 0.753 - ETA: 1:02:37 - loss: 0.5180 - accuracy: 0.753 - ETA: 1:02:36 - loss: 0.5178 - accuracy: 0.753 - ETA: 1:02:36 - loss: 0.5174 - accuracy: 0.754 - ETA: 1:02:35 - loss: 0.5173 - accuracy: 0.754 - ETA: 1:02:35 - loss: 0.5172 - accuracy: 0.754 - ETA: 1:02:34 - loss: 0.5170 - accuracy: 0.754 - ETA: 1:02:34 - loss: 0.5171 - accuracy: 0.754 - ETA: 1:02:33 - loss: 0.5169 - accuracy: 0.754 - ETA: 1:02:33 - loss: 0.5167 - accuracy: 0.754 - ETA: 1:02:32 - loss: 0.5167 - accuracy: 0.754 - ETA: 1:02:31 - loss: 0.5163 - accuracy: 0.754 - ETA: 1:02:31 - loss: 0.5160 - accuracy: 0.754 - ETA: 1:02:30 - loss: 0.5161 - accuracy: 0.754 - ETA: 1:02:30 - loss: 0.5159 - accuracy: 0.754 - ETA: 1:02:29 - loss: 0.5155 - accuracy: 0.755 - ETA: 1:02:29 - loss: 0.5152 - accuracy: 0.755 - ETA: 1:02:28 - loss: 0.5148 - accuracy: 0.7

...

6566/7233 [==========================>...] - ETA: 7:57 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:57 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:56 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:55 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:55 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:54 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:54 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:53 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:53 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:52 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:51 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:51 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:50 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:50 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:49 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:49 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:48 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:48 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:47 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:46 - loss: 0.2150 - accuracy: 0.91 - ETA: 7:46 - loss: 0.2151 - accuracy: 0.91 - ETA: 7:45 - loss: 0.2151 - accu

6752/7233 [===========================>..] - ETA: 6:13 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:12 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:12 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:11 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:11 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:10 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:09 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:09 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:08 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:08 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:07 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:07 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:06 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:06 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:05 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:04 - loss: 0.2155 - accuracy: 0.91 - ETA: 6:04 - loss: 0.2156 - accuracy: 0.91 - ETA: 6:03 - loss: 0.2156 - accuracy: 0.91 - ETA: 6:03 - loss: 0.2156 - accuracy: 0.91 - ETA: 6:02 - loss: 0.2156 - accuracy: 0.91 - ETA: 6:02 - loss: 0.2156 - accuracy: 0.91 - ETA: 6:01 - loss: 0.2156 - accu

6938/7233 [===========================>..] - ETA: 4:29 - loss: 0.2161 - accuracy: 0.91 - ETA: 4:28 - loss: 0.2161 - accuracy: 0.91 - ETA: 4:27 - loss: 0.2161 - accuracy: 0.91 - ETA: 4:27 - loss: 0.2161 - accuracy: 0.91 - ETA: 4:26 - loss: 0.2161 - accuracy: 0.91 - ETA: 4:26 - loss: 0.2161 - accuracy: 0.91 - ETA: 4:25 - loss: 0.2161 - accuracy: 0.91 - ETA: 4:25 - loss: 0.2161 - accuracy: 0.91 - ETA: 4:24 - loss: 0.2162 - accuracy: 0.91 - ETA: 4:24 - loss: 0.2162 - accuracy: 0.91 - ETA: 4:23 - loss: 0.2162 - accuracy: 0.91 - ETA: 4:22 - loss: 0.2162 - accuracy: 0.91 - ETA: 4:22 - loss: 0.2162 - accuracy: 0.91 - ETA: 4:21 - loss: 0.2162 - accuracy: 0.91 - ETA: 4:21 - loss: 0.2162 - accuracy: 0.91 - ETA: 4:20 - loss: 0.2163 - accuracy: 0.91 - ETA: 4:20 - loss: 0.2163 - accuracy: 0.91 - ETA: 4:19 - loss: 0.2162 - accuracy: 0.91 - ETA: 4:18 - loss: 0.2162 - accuracy: 0.91 - ETA: 4:18 - loss: 0.2162 - accuracy: 0.91 - ETA: 4:17 - loss: 0.2162 - accuracy: 0.91 - ETA: 4:17 - loss: 0.2162 - accu

7124/7233 [============================>.] - ETA: 2:44 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:44 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:43 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:43 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:42 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:41 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:41 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:40 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:40 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:39 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:39 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:38 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:38 - loss: 0.2165 - accuracy: 0.91 - ETA: 2:37 - loss: 0.2165 - accuracy: 0.91 - ETA: 2:36 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:36 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:35 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:35 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:34 - loss: 0.2164 - accuracy: 0.91 - ETA: 2:34 - loss: 0.2165 - accuracy: 0.91 - ETA: 2:33 - loss: 0.2165 - accuracy: 0.91 - ETA: 2:33 - loss: 0.2165 - accu

7233/7233 [==============================] - ETA: 1:00 - loss: 0.2172 - accuracy: 0.91 - ETA: 59s - loss: 0.2172 - accuracy: 0.9135 - ETA: 59s - loss: 0.2172 - accuracy: 0.913 - ETA: 58s - loss: 0.2172 - accuracy: 0.913 - ETA: 58s - loss: 0.2172 - accuracy: 0.913 - ETA: 57s - loss: 0.2173 - accuracy: 0.913 - ETA: 57s - loss: 0.2173 - accuracy: 0.913 - ETA: 56s - loss: 0.2172 - accuracy: 0.913 - ETA: 56s - loss: 0.2173 - accuracy: 0.913 - ETA: 55s - loss: 0.2173 - accuracy: 0.913 - ETA: 54s - loss: 0.2172 - accuracy: 0.913 - ETA: 54s - loss: 0.2172 - accuracy: 0.913 - ETA: 53s - loss: 0.2172 - accuracy: 0.913 - ETA: 53s - loss: 0.2172 - accuracy: 0.913 - ETA: 52s - loss: 0.2172 - accuracy: 0.913 - ETA: 52s - loss: 0.2172 - accuracy: 0.913 - ETA: 51s - loss: 0.2172 - accuracy: 0.913 - ETA: 51s - loss: 0.2172 - accuracy: 0.913 - ETA: 50s - loss: 0.2172 - accuracy: 0.913 - ETA: 49s - loss: 0.2172 - accuracy: 0.913 - ETA: 49s - loss: 0.2172 - accuracy: 0.913 - ETA: 48s - loss: 0.2172 - accu

## F1 : percentile

In [18]:
def extractAttentionModel(model,attention_layer_number = -6):
    return models.Model(inputs=model.input,outputs=model.layers[attention_layer_number].score.output)

In [19]:
attention_model = extractAttentionModel(model)

In [20]:
def assembleTokensWithAttentions(tokens,attention_vector):
    new_tokens = []
    temp_token = ""
    temp_score = 0
    temp_count = 0
    start = True
    concat = False
    for token,attention in zip(tokens,attention_vector):
        if "##" in token or token in punctuation or concat:
            temp_token += token[2:] if "##" in token else token
            temp_score += attention
            temp_count += 1
        else:
            if not start:
                if temp_count!=0:
                    new_tokens.append((temp_token,temp_score/temp_count))
                temp_token = token
                temp_score = attention
                temp_count = 1
            else:
                temp_token = token
                temp_score = attention
                temp_count = 1
                start = False
        concat = False
        if token in punctuation:
            concat = True
    if temp_count!=0:
        new_tokens.append((temp_token,temp_score/temp_count))
    return new_tokens

In [21]:
def getSlectedIndicesForTextWithBertAttention(text,attention,prediction,tokens,percentile_n,stopwords,remove_pos_sentiment=False,treshold=1e-4):
    indices = []
    selected_tokens = []
    if tf.argmax(prediction) == 1:
        tokens = assembleTokensWithAttentions(tokens,attention)
        attention = np.array([x[1] for x in tokens])
        if len(attention) == 0:
            return list(indices),selected_tokens
        att_max = np.max(attention)
        p = att_max - attention
        tresh = np.percentile(p,percentile_n)
        parent_start = 0
        start = 0
        for index,(token,score) in enumerate(tokens):
            start_index = max(text[start:].find(token),0)
            selected_tokens.append((token,score))
            if (att_max - score <= tresh) and score > treshold and token not in punctuation and token not in stopwords and (textblob.TextBlob(token).polarity <= 0 if remove_pos_sentiment else True):
                indices += list(range(start + start_index,start + start_index+len(token)))
            start += start_index+len(token)            
    prev = -10
    space_list = []
    for i in indices:
        if i - 2 == prev:
            space_list.append(i-1)
    indices = set(indices + space_list) 
    return list(indices),selected_tokens


In [22]:
def getSelectedIndicesForDatasetWithBertAttention(texts,original_texts,attentions,predictions,tokenizer,percentile_n,stopwords,remove_pos_sentiment=False,treshold=1e-4):
    indices_all,tokens_all = [],[]
    for text,original_text,attention,prediction in zip(texts,original_texts,attentions,predictions):
        tokens = tokenizer.tokenize(text)
        indices , tokens = getSlectedIndicesForTextWithBertAttention(original_text,attention,prediction,tokens,percentile_n,stopwords,remove_pos_sentiment,treshold)
        indices_all.append(indices)
        tokens_all.append(tokens)
    return indices_all,tokens_all

In [ ]:
# result on test dataset from civil
predictions = model.predict(test_data)
trues = tf.argmax(test_labels,axis=-1)
print(classification_report(trues,tf.argmax(predictions,axis=-1)))

              precision    recall  f1-score   support

           0       0.86      0.85      0.85     28988
           1       0.85      0.86      0.85     28872

    accuracy                           0.85     57860
   macro avg       0.85      0.85      0.85     57860
weighted avg       0.85      0.85      0.85     57860



In [23]:
test_data = clean_text(toxic_span_dataset['text'].to_list(),bert_tokenizer_instance)
input_ids,input_masks,input_segs = create_req_bert_inputs(test_data,bert_tokenizer_instance)
test_data = [np.array(input_ids),np.array(input_segs),np.array(input_masks)]

100%|██████████| 8629/8629 [00:00<00:00, 27492.49it/s]


In [24]:
# result on test dataset from toxic span
attentions = attention_model.predict(test_data)
predictions = model.predict(test_data)
trues = toxic_span_dataset['spans'].apply(lambda x : 0 if len(x)==0 else 1)
print(classification_report(trues,tf.argmax(predictions,axis=-1)))

              precision    recall  f1-score   support

           0       0.03      0.02      0.02       528
           1       0.94      0.96      0.95      8101

    accuracy                           0.91      8629
   macro avg       0.48      0.49      0.49      8629
weighted avg       0.88      0.91      0.89      8629



In [ ]:
results = []
for j in [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9,1e-10]:
    print("threshold = %f"%(j))
    res = []
    for i in range(1,100):
        toxic_indices,toxic_tokens = getSelectedIndicesForDatasetWithBertAttention(toxic_span_dataset['text'].to_numpy(),toxic_span_dataset['original_text'].to_numpy(),attentions,predictions,bert_tokenizer_instance,i,stopwords,treshold=j)
        toxic_f1 = avg_f1(toxic_indices,toxic_span_dataset['spans'].to_numpy())
        print("for n = %d: F1 = %f"%(i,toxic_f1))
        res.append(toxic_f1)
    results.append(res)
        

threshold = 0.010000
for n = 1: F1 = 0.561976
for n = 2: F1 = 0.566787
for n = 3: F1 = 0.570632
for n = 4: F1 = 0.574462
for n = 5: F1 = 0.576304
for n = 6: F1 = 0.578135
for n = 7: F1 = 0.579271
for n = 8: F1 = 0.580743
for n = 9: F1 = 0.581753
for n = 10: F1 = 0.583199
for n = 11: F1 = 0.583115
for n = 12: F1 = 0.583266
for n = 13: F1 = 0.583760
for n = 14: F1 = 0.583798
for n = 15: F1 = 0.584228
for n = 16: F1 = 0.584421
for n = 17: F1 = 0.585273
for n = 18: F1 = 0.585267
for n = 19: F1 = 0.585297
for n = 20: F1 = 0.585885
for n = 21: F1 = 0.585885
for n = 22: F1 = 0.585895
for n = 23: F1 = 0.586062
for n = 24: F1 = 0.586116
for n = 25: F1 = 0.586190
for n = 26: F1 = 0.586190
for n = 27: F1 = 0.586190
for n = 28: F1 = 0.586190
for n = 29: F1 = 0.586091
for n = 30: F1 = 0.586081
for n = 31: F1 = 0.586081
for n = 32: F1 = 0.586081
for n = 33: F1 = 0.586081
for n = 34: F1 = 0.586570
for n = 35: F1 = 0.586570
for n = 36: F1 = 0.586570
for n = 37: F1 = 0.586570
for n = 38: F1 = 0.586564


In [ ]:
fig,subplots = plt.subplots(2,5)
fig.set_size_inches(30,10)
for i in range(10):
    subplots[int(i/5),i%5].scatter(range(1,100),results[i])
    subplots[int(i/5),i%5].set_xlabel('percentile\n threshold = 1e-'+str(i+1))
    subplots[int(i/5),i%5].set_ylabel('F1')
plt.savefig('fig.png')

In [ ]:
def get_annotations(index):
    nums = sorted(set(index))
    gaps = [[s, e] for s, e in zip(nums, nums[1:]) if s+1 < e]
    edges = iter(nums[:1] + sum(gaps, []) + nums[-1:])
    return list(zip(edges, edges))

In [ ]:
spans = toxic_span_dataset.apply(lambda row : [(row['original_text'][np.min(span):np.max(span)+1]) for span in get_annotations(row['spans'])],axis=1).to_numpy()
toxic_indices,toxic_tokens = getSelectedIndicesForDatasetWithBertAttention(toxic_span_dataset['text'].to_numpy(),toxic_span_dataset['original_text'].to_numpy(),attentions,predictions,bert_tokenizer_instance,5,stopwords)
with open("answers.txt","w") as file:
    for tokens,span in zip(toxic_tokens,spans):
        file.write(str(tokens)+" "+str(span)+"\n")

## F1: percentile + remove positive sentiment 

In [25]:
for i in range(1,100):
    toxic_indices,toxic_tokens = getSelectedIndicesForDatasetWithBertAttention(toxic_span_dataset['text'].to_numpy(),toxic_span_dataset['original_text'].to_numpy(),attentions,predictions,bert_tokenizer_instance,i,stopwords,remove_pos_sentiment=True,treshold=1e-4)
    toxic_f1 = avg_f1(toxic_indices,toxic_span_dataset['spans'].to_numpy())
    print("for n = %d: F1 = %f"%(i,toxic_f1))

for n = 1: F1 = 0.563556
for n = 2: F1 = 0.570554
for n = 3: F1 = 0.576782
for n = 4: F1 = 0.582548
for n = 5: F1 = 0.585124
for n = 6: F1 = 0.587675
for n = 7: F1 = 0.589923
for n = 8: F1 = 0.592190
for n = 9: F1 = 0.593093
for n = 10: F1 = 0.595077
for n = 11: F1 = 0.594786
for n = 12: F1 = 0.595112
for n = 13: F1 = 0.596021
for n = 14: F1 = 0.595985
for n = 15: F1 = 0.596816
for n = 16: F1 = 0.596995
for n = 17: F1 = 0.598055
for n = 18: F1 = 0.598117
for n = 19: F1 = 0.598203
for n = 20: F1 = 0.599055
for n = 21: F1 = 0.599115
for n = 22: F1 = 0.599253
for n = 23: F1 = 0.599447
for n = 24: F1 = 0.599558
for n = 25: F1 = 0.600117
for n = 26: F1 = 0.600115
for n = 27: F1 = 0.600154
for n = 28: F1 = 0.600184
for n = 29: F1 = 0.600407
for n = 30: F1 = 0.600456
for n = 31: F1 = 0.600477
for n = 32: F1 = 0.600462
for n = 33: F1 = 0.600462
for n = 34: F1 = 0.601594
for n = 35: F1 = 0.601594
for n = 36: F1 = 0.601581
for n = 37: F1 = 0.601530
for n = 38: F1 = 0.601392
for n = 39: F1 = 0.60

## F1: percentile + remove positive sentiment +Keyword

In [26]:
hate_words = pd.read_csv("./bad-words.csv")['jigaboo'].to_list()

In [27]:
def getInidcesOfKeyWord(texts,hate_words):
    indices_all = []
    for text in texts:
        start = 0
        indices = []
        for word in word_tokenize(text):
            word_blob = textblob.TextBlob(word)
            if word_blob.polarity < 0 or word in hate_words:
                start_index = text[start:].find(word)
                indices += list(range(start + start_index,start + start_index+len(word)))
                start += start_index+len(word)
        indices_all.append(indices)
    return indices_all

In [ ]:
key_word_indices = getInidcesOfKeyWord(toxic_span_dataset['text'].to_numpy(),hate_words)
for i in range(1,100):
    indices = []
    toxic_indices,toxic_tokens = getSelectedIndicesForDatasetWithBertAttention(toxic_span_dataset['text'].to_numpy(),toxic_span_dataset['original_text'].to_numpy(),attentions,predictions,bert_tokenizer_instance,i,stopwords,remove_pos_sentiment=True,treshold=1e-4)
    for k,j in zip(toxic_indices,key_word_indices):
        indices.append(list(set(k).union(set(j))))
    toxic_f1 = avg_f1(indices,toxic_span_dataset['spans'].to_numpy())
    print("for n = %d: F1 = %f"%(i,toxic_f1))

for n = 1: F1 = 0.472626
for n = 2: F1 = 0.476972
for n = 3: F1 = 0.480882
for n = 4: F1 = 0.484782
for n = 5: F1 = 0.487026
for n = 6: F1 = 0.488674
for n = 7: F1 = 0.490034
for n = 8: F1 = 0.491510
for n = 9: F1 = 0.492050
for n = 10: F1 = 0.492912
for n = 11: F1 = 0.492831
for n = 12: F1 = 0.492861
for n = 13: F1 = 0.493442
for n = 14: F1 = 0.493452
for n = 15: F1 = 0.493731
for n = 16: F1 = 0.493945
for n = 17: F1 = 0.494618
for n = 18: F1 = 0.494696
for n = 19: F1 = 0.494781
for n = 20: F1 = 0.495153
for n = 21: F1 = 0.495198
for n = 22: F1 = 0.495274
for n = 23: F1 = 0.495441
for n = 24: F1 = 0.495528
for n = 25: F1 = 0.495796
for n = 26: F1 = 0.495794
for n = 27: F1 = 0.495778
for n = 28: F1 = 0.495741
for n = 29: F1 = 0.495843
for n = 30: F1 = 0.495880
for n = 31: F1 = 0.495882
for n = 32: F1 = 0.495869
for n = 33: F1 = 0.495869
for n = 34: F1 = 0.496512
for n = 35: F1 = 0.496512
for n = 36: F1 = 0.496498
for n = 37: F1 = 0.496455
for n = 38: F1 = 0.496344
for n = 39: F1 = 0.49

## F1 : Decision Tree

In [ ]:
hate_words = pd.read_csv("./bad-words.csv")['jigaboo'].to_list()

In [ ]:
stanza.download()
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

2020-12-12 14:38:36 INFO: Downloading default packages for language: en (English)...
INFO:stanza:Downloading default packages for language: en (English)...
2020-12-12 14:39:37 INFO: Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
2020-12-12 14:39:37 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-12-12 14:39:37 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

INFO:stanza:Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

2020-12-12 14:39:37 INFO: Use device: gpu
INFO:stanza:Use device: gpu
2020-12-12 14:39:37 INFO: Loading: tokenize
INFO:stanza:Loading: tokenize
2020-12-12 1

In [ ]:
def getDeprelDependancies(text,nlp):
    if text == "":
        return []
    doc = nlp(text)
    tokens = []
    for sent in doc.sentences:
        for word in sent.words:
            tokens.append((word.text,word.deprel,word.upos,sent.words[word.head-1].text))
    return tokens

In [ ]:
def getDeprelRoleOftoken(token,deps,start_index=0):
    temp = start_index
    for dep in deps[start_index:]:
        temp += 1
        if dep[0] == token:
            return dep[1],temp
    return None,start_index

In [ ]:
def mergeTokens(tokens):
    merged = []
    prev = ""
    start = True
    for token in tokens:
        if token[:2] == "##":
            prev += token[2:]
        elif start:
            prev = token
            start = False
        elif prev != "":
            merged.append(prev)
            prev = token
    if prev != "":
        merged.append(prev)
        prev = token
    return merged

In [ ]:
def makeDTInputForText(text,tokens,attention,deprel_roles_dict,hatewords,nlp):
    mergedTokens = mergeTokens(tokens)
    dependencies = getDeprelDependancies(text,nlp)
    i = -1
    deprel_index = 0
    result = []
    for token,att in zip(tokens,attention):
        if token[:2] != "##":
            i += 1
        hate = 1 if mergedTokens[i] in hatewords else 0
        sentiment = textblob.TextBlob(mergedTokens[i]).polarity
        deprel_dep,deprel_index = getDeprelRoleOftoken(mergedTokens[i],dependencies,deprel_index)
        result.append([hate,sentiment,att,deprel_roles_dict[deprel_dep]])
    return result

In [ ]:
def makeDTInputForDataset(texts,attentions,nlp,tokenizer,hatewords):
    inputs = []
    roles = [
        'acl','advcl','advmod','amod','appos','aux','case','cc','ccomp','clf','compound',
        'conj','cop','csubj','dep','det','discourse','dislocated','expl','fixed','flat',
        'goeswith','iobj','list','mark','nmod','nsubj','nummod','obj','obl','orphan','parataxis',
        'punct','reparandum','root','vocative','xcomp'
    ]
    roles_map = defaultdict(lambda : 0)
    for i,role in enumerate(roles):
        roles_map[role] = i+1
    for text,attention in zip(texts,attentions):
        tokens = tokenizer.tokenize(text)
        inputs += makeDTInputForText(text,tokens,attention,roles_map,hatewords,nlp)
    return inputs

In [ ]:
def makeDTOutPutForText(text,spans,tokens):
    output = []
    length = 0
    start = True
    prev = False
    for index,token in enumerate(tokens):
        sub = False
        if "##" in token:
            sub = True
            token = token[2:]
        if not start:
            next_index = text[length:].find(token)
            if next_index == 0:
                sub = True
            length += next_index
        if length in spans or (sub and prev):
            output.append(1)
            prev = True
        else:
            prev = False
            output.append(0)
        length += len(token)
        start = False
    return output

In [ ]:
def makeDTOutPutForDataset(texts,spans,tokenizer):
    outputs = []
    for text,span in zip(texts,spans):
        tokens = tokenizer.tokenize(text)
        outputs += makeDTOutPutForText(text,span,tokens)
    return outputs

In [ ]:
def getDTIndicesFortext(outputs,text,tokens):
    index = 0
    indexes = []
    sub = False
    prev = False
    for token,output in zip(tokens,outputs):
        if token[:2] == "##":
            token = token[2:]
            sub = True
        else:
            sub = False
        temp_index = text[index:].find(token)
        temp_start = index+temp_index
        if output == 1 or (sub and prev and output != 0):
            prev = True
            indexes = indexes + list(range(temp_start,temp_start+len(token)))
        else:
            prev = False
        index = temp_start+len(token)
    return indexes

In [ ]:
def getDTIndicesForDataset(outputs,texts,tokenizer):
    output_return = []
    for text,output in zip(texts,outputs):
        output_return.append(getDTIndicesFortext(output,text,tokenizer.tokenize(text)))
    return output_return

### train test split : Kfold

In [ ]:
texts = toxic_span_dataset['original_text'].to_numpy()

In [ ]:
input_data = np.array(makeDTInputForDataset(texts,attentions,nlp,bert_tokenizer_instance,hate_words))
output_data = np.array(makeDTOutPutForDataset(texts,toxic_span_dataset['spans'].to_numpy(),bert_tokenizer_instance))
result = []
for i in range(20,50,10):
    kf = KFold(n_splits=5)
    for train_index,test_index in kf.split(texts):
        x_train , x_test = input_data[train_index] , input_data[test_index]
        y_train , y_test = output_data[train_index] , output_data[test_index]
        dt = DecisionTreeClassifier(max_depth=i)
        dt.fit(x_train,y_train)
        preds = dt.predict(x_test)
        print("depth = %d"%i)
        print(classification_report(y_test,preds))
        preds = []
        for text,attention in zip(texts[test_index],attentions[test_index]):
            temp_data = np.array(makeDTInputForDataset([text],[attention],nlp,bert_tokenizer_instance,hate_words))
            preds.append(dt.predict(temp_data))
        indices = getDTIndicesForDataset(preds,texts[test_index],bert_tokenizer_instance)
        trues = toxic_span_dataset['spans'].to_numpy()[test_index]
        f1_toxic = avg_f1(indices,trues)
        result.append((i,f1_toxic))

depth = 20
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      1532
           1       0.53      0.12      0.20       194

    accuracy                           0.89      1726
   macro avg       0.72      0.56      0.57      1726
weighted avg       0.86      0.89      0.86      1726

depth = 20
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1613
           1       0.68      0.19      0.29       113

    accuracy                           0.94      1726
   macro avg       0.81      0.59      0.63      1726
weighted avg       0.93      0.94      0.92      1726

depth = 20
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      1640
           1       0.25      0.19      0.21        86

    accuracy                           0.93      1726
   macro avg       0.60      0.58      0.59      1726
weighted avg       0.92      0.93      0.

In [ ]:
result

[(20, 0.37082621910686786),
 (20, 0.3387159049096937),
 (20, 0.3594486888388913),
 (20, 0.3707629366648917),
 (20, 0.36115783278773184),
 (30, 0.37159722646520066),
 (30, 0.3383052661507055),
 (30, 0.3604294289301493),
 (30, 0.3639040661155896),
 (30, 0.36067299718713863),
 (40, 0.3705688888702539),
 (40, 0.3374397791907423),
 (40, 0.3591089590952622),
 (40, 0.36360457549088837),
 (40, 0.3618959249252813)]

In [ ]:
for i in range(20,50,10):
    f1_toxic = 0
    for j in range(5):
        f1_toxic += result[int((i-20)*0.5)+j][1]
    print("max_depth = %d : F1 = %f "%(i,f1_toxic/5))

max_depth = 20 : F1 = 0.360182 
max_depth = 30 : F1 = 0.358982 
max_depth = 40 : F1 = 0.358524 


## F1 : SVM

In [ ]:
hate_words = pd.read_csv("./bad-words.csv")['jigaboo'].to_list()

In [ ]:
stanza.download()
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

2020-12-12 16:40:17 INFO: Downloading default packages for language: en (English)...
INFO:stanza:Downloading default packages for language: en (English)...
2020-12-12 16:40:18 INFO: File exists: /root/stanza_resources/en/default.zip.
INFO:stanza:File exists: /root/stanza_resources/en/default.zip.
2020-12-12 16:40:22 INFO: Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
2020-12-12 16:40:22 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-12-12 16:40:22 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

INFO:stanza:Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

2020-12-12 16:

In [ ]:
def getDeprelDependancies(text,nlp):
    if text == "":
        return []
    doc = nlp(text)
    tokens = []
    for sent in doc.sentences:
        for word in sent.words:
            tokens.append((word.text,word.deprel,word.upos,sent.words[word.head-1].text))
    return tokens

In [ ]:
def getDeprelRoleOftoken(token,deps,start_index=0):
    temp = start_index
    for dep in deps[start_index:]:
        temp += 1
        if dep[0] == token:
            return dep[1],temp
    return None,start_index

In [ ]:
def mergeTokens(tokens):
    merged = []
    prev = ""
    start = True
    for token in tokens:
        if token[:2] == "##":
            prev += token[2:]
        elif start:
            prev = token
            start = False
        elif prev != "":
            merged.append(prev)
            prev = token
    if prev != "":
        merged.append(prev)
        prev = token
    return merged

In [ ]:
def makeSVMInputForText(text,tokens,attention,deprel_roles_dict,hatewords,nlp):
    mergedTokens = mergeTokens(tokens)
    dependencies = getDeprelDependancies(text,nlp)
    i = -1
    deprel_index = 0
    result = []
    for token,att in zip(tokens,attention):
        if token[:2] != "##":
            i += 1
        hate = 1 if mergedTokens[i] in hatewords else 0
        sentiment = textblob.TextBlob(mergedTokens[i]).polarity
        deprel_dep,deprel_index = getDeprelRoleOftoken(mergedTokens[i],dependencies,deprel_index)
        result.append([hate,sentiment,att,deprel_roles_dict[deprel_dep]])
    return result

In [ ]:
def makeSVMInputForDataset(texts,attentions,nlp,tokenizer,hatewords):
    inputs = []
    roles = [
        'acl','advcl','advmod','amod','appos','aux','case','cc','ccomp','clf','compound',
        'conj','cop','csubj','dep','det','discourse','dislocated','expl','fixed','flat',
        'goeswith','iobj','list','mark','nmod','nsubj','nummod','obj','obl','orphan','parataxis',
        'punct','reparandum','root','vocative','xcomp'
    ]
    roles_map = defaultdict(lambda : 0)
    for i,role in enumerate(roles):
        roles_map[role] = i+1
    for text,attention in zip(texts,attentions):
        tokens = tokenizer.tokenize(text)
        inputs += makeSVMInputForText(text,tokens,attention,roles_map,hatewords,nlp)
    return inputs

In [ ]:
def makeSVMOutPutForText(text,spans,tokens):
    output = []
    length = 0
    start = True
    for index,token in enumerate(tokens):
        sub = False
        if "##" in token:
            sub = True
            token = token[2:]
        if not start:
            next_index = text[length:].find(token)
            if next_index == 0:
                sub = True
            length += next_index
        if length in spans:
            output.append(1)
        else:
            output.append(0)
        length += len(token)
        start = False
    return output

In [ ]:
def makeSVMOutPutForDataset(texts,spans,tokenizer):
    outputs = []
    for text,span in zip(texts,spans):
        tokens = tokenizer.tokenize(text)
        outputs += makeSVMOutPutForText(text,span,tokens)
    return outputs

In [ ]:
def getSVMIndicesFortext(outputs,text,tokens):
    index = 0
    indexes = []
    sub = False
    prev = False
    for token,output in zip(tokens,outputs):
        if token[:2] == "##":
            token = token[2:]
            sub = True
        else:
            sub = False
        temp_index = text[index:].find(token)
        temp_start = index+temp_index
        if output == 1 or (sub and prev and output != 0):
            prev = True
            indexes = indexes + list(range(temp_start,temp_start+len(token)))
        else:
            prev = False
        index = temp_start+len(token)
    return indexes

In [ ]:
def getSVMIndicesForDataset(outputs,texts,tokenizer):
    output_ret = []
    for text,output in zip(texts,outputs):
        output_ret.append(getDTIndicesFortext(output,text,tokenizer.tokenize(text)))
    return output_ret

### train test split : Kfold

In [ ]:
input_data = np.array(makeSVMInputForDataset(texts,attentions,nlp,bert_tokenizer_instance,hate_words))
output_data = np.array(makeSVMOutPutForDataset(texts,toxic_span_dataset['spans'].to_numpy(),bert_tokenizer_instance))
result = []
kf = KFold(n_splits=5)
for train_index,test_index in kf.split(texts):
    x_train , x_test = input_data[train_index] , input_data[test_index]
    y_train , y_test = output_data[train_index] , output_data[test_index]
    dt = SVC()
    dt.fit(x_train,y_train)
    preds = dt.predict(x_test)
    print(classification_report(y_test,preds))
    preds = []
    for text,attention in zip(texts[test_index],attentions[test_index]):
        temp_data = np.array(makeDTInputForDataset([text],[attention],nlp,bert_tokenizer_instance,hate_words))
        preds.append(dt.predict(temp_data))
    indices = getSVMIndicesForDataset(preds,texts[test_index],bert_tokenizer_instance)
    trues = toxic_span_dataset['spans'].to_numpy()[test_index]
    f1_toxic = avg_f1(indices,trues)
    result.append(f1_toxic)
f1_toxic = 0
for j in range(5):
    f1_toxic += result[j]
print("F1 = %f "%(f1_toxic/5))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1555
           1       0.00      0.00      0.00       171

    accuracy                           0.90      1726
   macro avg       0.45      0.50      0.47      1726
weighted avg       0.81      0.90      0.85      1726



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.98      1644
           1       0.00      0.00      0.00        82

    accuracy                           0.95      1726
   macro avg       0.48      0.50      0.49      1726
weighted avg       0.91      0.95      0.93      1726



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1665
           1       0.00      0.00      0.00        61

    accuracy                           0.96      1726
   macro avg       0.48      0.50      0.49      1726
weighted avg       0.93      0.96      0.95      1726



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1625
           1       0.00      0.00      0.00       101

    accuracy                           0.94      1726
   macro avg       0.47      0.50      0.48      1726
weighted avg       0.89      0.94      0.91      1726



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1630
           1       0.00      0.00      0.00        95

    accuracy                           0.94      1725
   macro avg       0.47      0.50      0.49      1725
weighted avg       0.89      0.94      0.92      1725

F1 = 0.061188 


## F1 : Neural Network as Critic

In [29]:
hate_words = pd.read_csv("./bad-words.csv")['jigaboo'].to_list()

In [30]:
stanza.download()
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

2020-12-15 10:44:40 INFO: Downloading default packages for language: en (English)...
INFO:stanza:Downloading default packages for language: en (English)...
2020-12-15 10:45:45 INFO: Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
2020-12-15 10:45:45 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-12-15 10:45:45 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

INFO:stanza:Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

2020-12-15 10:45:45 INFO: Use device: gpu
INFO:stanza:Use device: gpu
2020-12-15 10:45:46 INFO: Loading: tokenize
INFO:stanza:Loading: tokenize
2020-12-15 1

In [31]:
def getDeprelDependancies(text,nlp):
    if text == "":
        return []
    doc = nlp(text)
    tokens = []
    for sent in doc.sentences:
        for word in sent.words:
            tokens.append((word.text,word.deprel,word.upos,sent.words[word.head-1].text))
    return tokens

In [32]:
def getDeprelRoleOftoken(token,deps,start_index=0):
    temp = start_index
    for dep in deps[start_index:]:
        temp += 1
        if dep[0] == token:
            return dep[1],temp
    return None,start_index

In [33]:
def getSlectedIndicesForTextWithBertAttentionWhenUseNNAsCritic(text,attention,prediction,tokens,percentile_n,stopwords,remove_pos_sentiment=False):
    indices = []
    selected_tokens = []
    if tf.argmax(prediction,axis=-1) == 1:
        tokens = assembleTokensWithAttentions(tokens,attention)
        attention = np.array([x[1] for x in tokens])
        if len(attention) == 0:
            return list(indices),selected_tokens
        att_max = np.max(attention)
        p = att_max - attention
        tresh = np.percentile(p,percentile_n)
        parent_start = 0
        start = 0
        for index,(token,score) in enumerate(tokens):
            start_index = max(text[start:].find(token),0)
            if (att_max - score <= tresh) and score > 1e-8 and token not in punctuation and token not in stopwords and (textblob.TextBlob(token).polarity <= 0 if remove_pos_sentiment else True):
                indices += list(range(start + start_index,start + start_index+len(token)))
                selected_tokens.append(index)
            start += start_index+len(token)            
    prev = -10
    space_list = []
    for i in indices:
        if i - 2 == prev:
            space_list.append(i-1)
    indices = set(indices + space_list) 
    return list(indices),selected_tokens


In [34]:
def getSelectedIndicesForDatasetWithBertAttention(texts,original_texts,attentions,predictions,tokenizer,percentile_n,stopwords,remove_pos_sentiment=False):
    indices_all,tokens_all = [],[]
    for text,original_text,attention,prediction in zip(texts,original_texts,attentions,predictions):
        tokens = tokenizer.tokenize(text)
        indices , tokens = getSlectedIndicesForTextWithBertAttention(original_text,attention,prediction,tokens,percentile_n,stopwords,remove_pos_sentiment)
        indices_all.append(indices)
        tokens_all.append(tokens)
    return indices_all,tokens_all

In [35]:
def checkIFTokenIsToxic(token,text,spans,index=0):
    find = text[index:].find(token)
    if find == -1:
        return 0,index
    else:
        if find in spans:
            return 1,find + index + len(token)
        else:
            return 0,find + index + len(token)

In [36]:
def getTokenIndicesIntext(token,text,index=0):
    find = text[index:].find(token)
    if find == -1:
        return [],index
    else:
        return list(range(find + index,find + index + len(token))),find + index + len(token)

In [37]:
def getNeuralNetworkOutputForText(tokens,text,toxic_index_list,spans,window_size=2):
    outs = []
    for i in toxic_index_list:
        output = []
        index = 0
        for j in range(max(0,i-window_size),min(i+window_size+1,len(tokens))):
            token , att = tokens[j]
            toxic,index = checkIFTokenIsToxic(token,text,spans,index)
            if toxic:
                output.append(1)
            else:
                output.append(0)
        if i < window_size:
            for j in range(0,window_size-i):
                output.append(output[0])
        if i >= len(tokens) - window_size:
            for j in range(0,i-(len(tokens) - window_size)+1):
                output.append(output[-1])
        outs.append(output)
    return outs

In [38]:
def getNeuralNetworkInputForText(tokens,hate_words,token_deps,deprel_roles_map,index_list,window_size=2):
    ret_attentions = []
    ret_hates = []
    ret_deps = []
    ret_sentiment = []
    for i in index_list:
        temp_attentions = []
        temp_hates = []
        temp_deps = []
        temp_sentiment = []
        index = max(0,i-window_size)
        index_upos = max(0,i-window_size)
        for j in range(max(0,i-window_size),min(i+window_size+1,len(tokens))):
            token , att = tokens[j]
            temp_attentions.append(att)
            hate = 1.0 if token in hate_words else 0.0
            temp_hates.append(hate)
            role,index = getDeprelRoleOftoken(token,token_deps,index)
            temp_deps.append(deprel_roles_map[role])
            temp_sentiment.append((TextBlob(token).sentiment.polarity+1.0)/2.0)
        if i < window_size:
            for j in range(0,window_size-i):
                temp_attentions.insert(0,temp_attentions[0])
                temp_hates.insert(0,temp_hates[0])
                temp_deps.insert(0,temp_deps[0])
                temp_sentiment.insert(0,temp_sentiment[0])
        if i >= len(tokens) - window_size:
            for j in range(0,i-(len(tokens) - window_size)+1):
                temp_attentions.append(temp_attentions[-1])
                temp_hates.append(temp_hates[-1])
                temp_deps.append(temp_deps[-1])
                temp_sentiment.append(temp_sentiment[-1])
        ret_attentions.append(np.array(temp_attentions))
        ret_hates.append(np.array(temp_hates))
        ret_deps.append(np.array(temp_deps))
        ret_sentiment.append(np.array(temp_sentiment))
    return ret_attentions,ret_hates,ret_deps,ret_sentiment

In [39]:
def getNeuralNetworkInputOutputForDataset(texts,original_texts,attentions,predictions,hate_words,stopwords,tokenizer,nlp,return_output=False,spans=None):
    ret_attentions = []
    ret_hates = []
    ret_deps = []
    ret_sentiment = []
    outputs = []
    toxic_terms_indices = []
    roles = [
        'acl','advcl','advmod','amod','appos','aux','case','cc','ccomp','clf','compound',
        'conj','cop','csubj','dep','det','discourse','dislocated','expl','fixed','flat',
        'goeswith','iobj','list','mark','nmod','nsubj','nummod','obj','obl','orphan','parataxis',
        'punct','reparandum','root','vocative','xcomp'
    ]
    deprel_roles_map = defaultdict(lambda : 0)
    for i,role in enumerate(roles):
        deprel_roles_map[role] = i+1
    for index,(text,org_text,att,preds) in enumerate(zip(texts,original_texts,attentions,predictions)):
        if text == "":
            continue
        tokens = tokenizer.tokenize(text)
        new_tokens = assembleTokensWithAttentions(tokens,att)
        if new_tokens == []:
            continue
        token_deps = getDeprelDependancies(text,nlp)
        toxic_char_index,toxic_terms_index = getSlectedIndicesForTextWithBertAttentionWhenUseNNAsCritic(org_text,att,preds,tokens,5,stopwords)
        inp_attentions,inp_hates,inp_deps,inp_sentiment = getNeuralNetworkInputForText(new_tokens,hate_words,token_deps,deprel_roles_map,toxic_terms_index,window_size=2)
        if inp_attentions == []:
            continue
        toxic_terms_indices.append(toxic_terms_index)
        ret_attentions = ret_attentions + inp_attentions
        ret_hates = ret_hates + inp_hates
        ret_deps = ret_deps + inp_deps
        ret_sentiment = ret_sentiment + inp_sentiment
        if return_output:
            outputs = outputs + getNeuralNetworkOutputForText(new_tokens,org_text,toxic_terms_index,spans[index],window_size=2)
    if return_output:
            return [np.array(ret_attentions),np.array(ret_deps),np.array(ret_hates),np.array(ret_sentiment)],utils.to_categorical(outputs)        
    return [np.array(ret_attentions),np.array(ret_deps),np.array(ret_hates),np.array(ret_sentiment)],toxic_terms_indices

In [40]:
def getNeuralNetworkOutputForDataset(texts,attentions,original_texts,spans,toxic_term_indices,tokenizer):
    outputs = []
    for text,org_text,att,span,toxic_term in zip(texts,original_texts,attentions,spans,toxic_term_indices):
        tokens = tokenizer.tokenize(text)
        tokens = assembleTokensWithAttentions(tokens,att)
        if tokens == [] or text == "":
            continue
        outs = getNeuralNetworkOutputForText(tokens,org_text,toxic_term,span,window_size=2)
        outputs = outputs + outs
    return outputs

In [41]:
def getIndicesForNeuralNetworkCriticForText(text,tokens,toxic_indices):
    inidces = []
    toxic_indices.sort()
    index = 0
    for i in toxic_indices:
        temp,index = getTokenIndicesIntext(tokens[i][0],text,index)
        inidces = inidces + temp
    return list(set(inidces))

In [42]:
def getIndicesForNeuralNetworkCriticForDataset(texts,attentions,preds_attention,original_texts,stopwords,hate_words,tokenizer,model,nlp):
    indices = []
    roles = [
        'acl','advcl','advmod','amod','appos','aux','case','cc','ccomp','clf','compound',
        'conj','cop','csubj','dep','det','discourse','dislocated','expl','fixed','flat',
        'goeswith','iobj','list','mark','nmod','nsubj','nummod','obj','obl','orphan','parataxis',
        'punct','reparandum','root','vocative','xcomp'
    ]
    deprel_roles_map = defaultdict(lambda : 0)
    for i,role in enumerate(roles):
        deprel_roles_map[role] = i+1
    for text,org_text,att,pred_att in zip(texts,original_texts,attentions,preds_attention):
        tokens = tokenizer.tokenize(text)
        token_deps = getDeprelDependancies(text,nlp)
        indices_toxic_att,selected_indices = getSlectedIndicesForTextWithBertAttentionWhenUseNNAsCritic(text,att,pred_att,tokens,5,stopwords)
        tokens = assembleTokensWithAttentions(tokens,att)
        selected_indices_new = []
        if len(selected_indices) != 0:
            ret_attentions,ret_hates,ret_deps,ret_sentiment = getNeuralNetworkInputForText(tokens,hate_words,token_deps,deprel_roles_map,selected_indices,window_size=2)
            data = [np.array(ret_attentions),np.array(ret_deps),np.array(ret_hates),np.array(ret_sentiment)]
            preds_nn = model.predict(data)
            preds_nn = tf.argmax(preds_nn,axis=-1)
            for each_pred_nn,index in zip(preds_nn,selected_indices):
                for i in range(-2,3):
                    if each_pred_nn[2+i] == 1:
                        if 0 <= index + i < len(tokens):
                            selected_indices_new.append(index + i)
        indices.append(list(set(getIndicesForNeuralNetworkCriticForText(org_text,tokens,selected_indices_new)).union(indices_toxic_att)))
    return indices

## define model

In [43]:
def createLSTMModelForToxicSelection(length,num_of_deprel_roles):
    input_attention_score = layers.Input(shape=(length*2+1,),name="attention_score")
    # input_words = layers.Input(shape=(length*2+1,word_embed_size),name="sub_sent")
    input_deps = layers.Input(shape=(length*2+1,),name="deps",dtype=tf.int32)
    input_hates = layers.Input(shape=(length*2+1,),name="hates")
    input_sentiments = layers.Input(shape=(length*2+1,),name="sentiments")
    dep_embed = layers.Embedding(num_of_deprel_roles,12)(input_deps)
    output = layers.Concatenate()([K.repeat_elements(K.expand_dims(input_attention_score),rep=6,axis=-1),K.expand_dims(input_hates),K.expand_dims(input_sentiments),dep_embed])
    output = layers.LSTM(1024,return_sequences=True)(output)
    output = layers.Dense(1024,activation="relu")(output)
    output = layers.Dense(512,activation="relu")(output)
    output = layers.Dense(2,activation="softmax")(output)
    model = models.Model(inputs=[input_attention_score,input_deps,input_hates,input_sentiments],outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),loss="categorical_crossentropy",metrics=['accuracy'])
    return model

## train

In [44]:
texts = toxic_span_dataset['text'].to_numpy()
originial_texts = toxic_span_dataset['original_text'].to_numpy()
all_spans = toxic_span_dataset['spans'].to_numpy()
result_test = []
result_train = []
kf = KFold(n_splits=5)
train_test_indices = []
for train_index,test_index in kf.split(texts):
    train_test_indices.append((train_index,test_index))

In [45]:
train_index,test_index = train_test_indices.pop()
train_texts,test_texts = texts[train_index],texts[test_index]
train_org_text,test_org_text = originial_texts[train_index],originial_texts[test_index]
train_attentions,test_attentions = attentions[train_index],attentions[test_index]
train_attmodel_preds,test_attmodel_preds = predictions[train_index],predictions[test_index]
train_spans,test_spans = all_spans[train_index],all_spans[test_index]
train_nn_data,train_nn_output = getNeuralNetworkInputOutputForDataset(train_texts,train_org_text,train_attentions,train_attmodel_preds,hate_words,stopwords,bert_tokenizer_instance,nlp,return_output=True,spans=train_spans)
test_nn_data,test_nn_output = getNeuralNetworkInputOutputForDataset(test_texts,test_org_text,test_attentions,test_attmodel_preds,hate_words,stopwords,bert_tokenizer_instance,nlp,return_output=True,spans=test_spans)
model = createLSTMModelForToxicSelection(2,38)
model.fit(train_nn_data,train_nn_output,validation_data=(test_nn_data,test_nn_output),batch_size=16,epochs=10)

Epoch 1/10
673/673 [==============================] - 6s 8ms/step - loss: 0.2990 - accuracy: 0.9125 - val_loss: 0.2822 - val_accuracy: 0.9165
Epoch 2/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2843 - accuracy: 0.9154 - val_loss: 0.2788 - val_accuracy: 0.9161
Epoch 3/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2798 - accuracy: 0.9166 - val_loss: 0.2741 - val_accuracy: 0.9165
Epoch 4/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2779 - accuracy: 0.9173 - val_loss: 0.2732 - val_accuracy: 0.9181
Epoch 5/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2757 - accuracy: 0.9173 - val_loss: 0.2711 - val_accuracy: 0.9179
Epoch 6/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2735 - accuracy: 0.9181 - val_loss: 0.2699 - val_accuracy: 0.9185
Epoch 7/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2729 - accuracy: 0.9176 - val_loss: 0.2803 - val_accuracy: 0.9190
Epoch 

In [46]:
test_inidces = getIndicesForNeuralNetworkCriticForDataset(test_texts,test_attentions,test_attmodel_preds,test_org_text,stopwords,hate_words,bert_tokenizer_instance,model,nlp)
toxic_f1 = avg_f1(test_inidces,test_spans)
print("F1 = %f"%(toxic_f1))
result_test.append(toxic_f1)

F1 = 0.353087


In [47]:
train_index,test_index = train_test_indices.pop()
train_texts,test_texts = texts[train_index],texts[test_index]
train_org_text,test_org_text = originial_texts[train_index],originial_texts[test_index]
train_attentions,test_attentions = attentions[train_index],attentions[test_index]
train_attmodel_preds,test_attmodel_preds = predictions[train_index],predictions[test_index]
train_spans,test_spans = all_spans[train_index],all_spans[test_index]
train_nn_data,train_nn_output = getNeuralNetworkInputOutputForDataset(train_texts,train_org_text,train_attentions,train_attmodel_preds,hate_words,stopwords,bert_tokenizer_instance,nlp,return_output=True,spans=train_spans)
test_nn_data,test_nn_output = getNeuralNetworkInputOutputForDataset(test_texts,test_org_text,test_attentions,test_attmodel_preds,hate_words,stopwords,bert_tokenizer_instance,nlp,return_output=True,spans=test_spans)
model = createLSTMModelForToxicSelection(2,38)
model.fit(train_nn_data,train_nn_output,validation_data=(test_nn_data,test_nn_output),batch_size=16,epochs=10)

Epoch 1/10
673/673 [==============================] - 6s 8ms/step - loss: 0.2991 - accuracy: 0.9113 - val_loss: 0.2770 - val_accuracy: 0.9169
Epoch 2/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2846 - accuracy: 0.9142 - val_loss: 0.2722 - val_accuracy: 0.9187
Epoch 3/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2814 - accuracy: 0.9155 - val_loss: 0.2775 - val_accuracy: 0.9182
Epoch 4/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2780 - accuracy: 0.9169 - val_loss: 0.2795 - val_accuracy: 0.9195
Epoch 5/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2766 - accuracy: 0.9167 - val_loss: 0.2647 - val_accuracy: 0.9215
Epoch 6/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2757 - accuracy: 0.9166 - val_loss: 0.2704 - val_accuracy: 0.9211
Epoch 7/10
673/673 [==============================] - 5s 8ms/step - loss: 0.2743 - accuracy: 0.9174 - val_loss: 0.2670 - val_accuracy: 0.9206
Epoch 

In [48]:
test_inidces = getIndicesForNeuralNetworkCriticForDataset(test_texts,test_attentions,test_attmodel_preds,test_org_text,stopwords,hate_words,bert_tokenizer_instance,model,nlp)
toxic_f1 = avg_f1(test_inidces,test_spans)
print("F1 = %f"%(toxic_f1))
result_test.append(toxic_f1)

F1 = 0.347260


In [49]:
train_index,test_index = train_test_indices.pop()
train_texts,test_texts = texts[train_index],texts[test_index]
train_org_text,test_org_text = originial_texts[train_index],originial_texts[test_index]
train_attentions,test_attentions = attentions[train_index],attentions[test_index]
train_attmodel_preds,test_attmodel_preds = predictions[train_index],predictions[test_index]
train_spans,test_spans = all_spans[train_index],all_spans[test_index]
train_nn_data,train_nn_output = getNeuralNetworkInputOutputForDataset(train_texts,train_org_text,train_attentions,train_attmodel_preds,hate_words,stopwords,bert_tokenizer_instance,nlp,return_output=True,spans=train_spans)
test_nn_data,test_nn_output = getNeuralNetworkInputOutputForDataset(test_texts,test_org_text,test_attentions,test_attmodel_preds,hate_words,stopwords,bert_tokenizer_instance,nlp,return_output=True,spans=test_spans)
model = createLSTMModelForToxicSelection(2,38)
model.fit(train_nn_data,train_nn_output,validation_data=(test_nn_data,test_nn_output),batch_size=16,epochs=10)

Epoch 1/10
672/672 [==============================] - 6s 8ms/step - loss: 0.2924 - accuracy: 0.9155 - val_loss: 0.3007 - val_accuracy: 0.9052
Epoch 2/10
672/672 [==============================] - 5s 8ms/step - loss: 0.2775 - accuracy: 0.9183 - val_loss: 0.3023 - val_accuracy: 0.9068
Epoch 3/10
672/672 [==============================] - 5s 7ms/step - loss: 0.2736 - accuracy: 0.9191 - val_loss: 0.2972 - val_accuracy: 0.9088
Epoch 4/10
672/672 [==============================] - 5s 8ms/step - loss: 0.2719 - accuracy: 0.9191 - val_loss: 0.2997 - val_accuracy: 0.9079
Epoch 5/10
672/672 [==============================] - 5s 8ms/step - loss: 0.2698 - accuracy: 0.9202 - val_loss: 0.2964 - val_accuracy: 0.9080
Epoch 6/10
672/672 [==============================] - 5s 8ms/step - loss: 0.2676 - accuracy: 0.9204 - val_loss: 0.3083 - val_accuracy: 0.9082
Epoch 7/10
672/672 [==============================] - 5s 8ms/step - loss: 0.2678 - accuracy: 0.9202 - val_loss: 0.2988 - val_accuracy: 0.9084
Epoch 

In [50]:
test_inidces = getIndicesForNeuralNetworkCriticForDataset(test_texts,test_attentions,test_attmodel_preds,test_org_text,stopwords,hate_words,bert_tokenizer_instance,model,nlp)
toxic_f1 = avg_f1(test_inidces,test_spans)
print("F1 = %f"%(toxic_f1))
result_test.append(toxic_f1) 

F1 = 0.363253


In [51]:
train_index,test_index = train_test_indices.pop()
train_texts,test_texts = texts[train_index],texts[test_index]
train_org_text,test_org_text = originial_texts[train_index],originial_texts[test_index]
train_attentions,test_attentions = attentions[train_index],attentions[test_index]
train_attmodel_preds,test_attmodel_preds = predictions[train_index],predictions[test_index]
train_spans,test_spans = all_spans[train_index],all_spans[test_index]
train_nn_data,train_nn_output = getNeuralNetworkInputOutputForDataset(train_texts,train_org_text,train_attentions,train_attmodel_preds,hate_words,stopwords,bert_tokenizer_instance,nlp,return_output=True,spans=train_spans)
test_nn_data,test_nn_output = getNeuralNetworkInputOutputForDataset(test_texts,test_org_text,test_attentions,test_attmodel_preds,hate_words,stopwords,bert_tokenizer_instance,nlp,return_output=True,spans=test_spans)
model = createLSTMModelForToxicSelection(2,38)
model.fit(train_nn_data,train_nn_output,validation_data=(test_nn_data,test_nn_output),batch_size=16,epochs=10)

Epoch 1/10
676/676 [==============================] - 6s 8ms/step - loss: 0.2983 - accuracy: 0.9119 - val_loss: 0.2720 - val_accuracy: 0.9181
Epoch 2/10
676/676 [==============================] - 5s 8ms/step - loss: 0.2856 - accuracy: 0.9135 - val_loss: 0.2781 - val_accuracy: 0.9195
Epoch 3/10
676/676 [==============================] - 5s 7ms/step - loss: 0.2805 - accuracy: 0.9155 - val_loss: 0.2672 - val_accuracy: 0.9208
Epoch 4/10
676/676 [==============================] - 5s 8ms/step - loss: 0.2775 - accuracy: 0.9162 - val_loss: 0.2694 - val_accuracy: 0.9197
Epoch 5/10
676/676 [==============================] - 5s 8ms/step - loss: 0.2763 - accuracy: 0.9167 - val_loss: 0.2654 - val_accuracy: 0.9214
Epoch 6/10
676/676 [==============================] - 5s 8ms/step - loss: 0.2746 - accuracy: 0.9178 - val_loss: 0.2639 - val_accuracy: 0.9221
Epoch 7/10
676/676 [==============================] - 5s 8ms/step - loss: 0.2746 - accuracy: 0.9172 - val_loss: 0.2645 - val_accuracy: 0.9212
Epoch 

In [52]:
test_inidces = getIndicesForNeuralNetworkCriticForDataset(test_texts,test_attentions,test_attmodel_preds,test_org_text,stopwords,hate_words,bert_tokenizer_instance,model,nlp)
toxic_f1 = avg_f1(test_inidces,test_spans)
print("F1 = %f"%(toxic_f1))
result_test.append(toxic_f1)

F1 = 0.350156


In [53]:
train_index,test_index = train_test_indices.pop()
train_texts,test_texts = texts[train_index],texts[test_index]
train_org_text,test_org_text = originial_texts[train_index],originial_texts[test_index]
train_attentions,test_attentions = attentions[train_index],attentions[test_index]
train_attmodel_preds,test_attmodel_preds = predictions[train_index],predictions[test_index]
train_spans,test_spans = all_spans[train_index],all_spans[test_index]
train_nn_data,train_nn_output = getNeuralNetworkInputOutputForDataset(train_texts,train_org_text,train_attentions,train_attmodel_preds,hate_words,stopwords,bert_tokenizer_instance,nlp,return_output=True,spans=train_spans)
test_nn_data,test_nn_output = getNeuralNetworkInputOutputForDataset(test_texts,test_org_text,test_attentions,test_attmodel_preds,hate_words,stopwords,bert_tokenizer_instance,nlp,return_output=True,spans=test_spans)
model = createLSTMModelForToxicSelection(2,38)
model.fit(train_nn_data,train_nn_output,validation_data=(test_nn_data,test_nn_output),batch_size=16,epochs=10)

Epoch 1/10
676/676 [==============================] - 6s 8ms/step - loss: 0.2999 - accuracy: 0.9122 - val_loss: 0.2853 - val_accuracy: 0.9144
Epoch 2/10
676/676 [==============================] - 5s 8ms/step - loss: 0.2846 - accuracy: 0.9145 - val_loss: 0.2733 - val_accuracy: 0.9165
Epoch 3/10
676/676 [==============================] - 5s 8ms/step - loss: 0.2796 - accuracy: 0.9157 - val_loss: 0.2722 - val_accuracy: 0.9189
Epoch 4/10
676/676 [==============================] - 5s 7ms/step - loss: 0.2777 - accuracy: 0.9167 - val_loss: 0.2791 - val_accuracy: 0.9156
Epoch 5/10
676/676 [==============================] - 5s 8ms/step - loss: 0.2749 - accuracy: 0.9167 - val_loss: 0.2822 - val_accuracy: 0.9144
Epoch 6/10
676/676 [==============================] - 5s 8ms/step - loss: 0.2751 - accuracy: 0.9168 - val_loss: 0.2721 - val_accuracy: 0.9175
Epoch 7/10
676/676 [==============================] - 5s 8ms/step - loss: 0.2721 - accuracy: 0.9175 - val_loss: 0.2756 - val_accuracy: 0.9178
Epoch 

In [54]:
test_inidces = getIndicesForNeuralNetworkCriticForDataset(test_texts,test_attentions,test_attmodel_preds,test_org_text,stopwords,hate_words,bert_tokenizer_instance,model,nlp)
toxic_f1 = avg_f1(test_inidces,test_spans)
print("F1 = %f"%(toxic_f1))
result_test.append(toxic_f1)

F1 = 0.358340


In [55]:
print("test F1 = %f"%(sum(result_test)/5.0))

test F1 = 0.354419
